# Road Following Live (with camera) - Inference Only

Load the optimized model (created with the [`optimize_model.ipynb` notebook](./optimize_model.ipynb)) executing the cell below

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('model_08-06_trt.pth'))

<All keys matched successfully>

Create the camera class.

In [2]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

In [3]:
import cv2
import ipywidgets
import threading

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

Create widgets to display the inference results

In [4]:
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
x_output_label = widgets.Label(value='X Value: 0.0')
y_output_label = widgets.Label(value='Y Value: 0.0')

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       x_output_label,
                       y_output_label], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         live_execution_widget]
    )
)

Execute the cell below to run inference and display the results.
A blue circle will show where the model thinks we should steer.

In [5]:
from utils import preprocess
from jetcam.utils import bgr8_to_jpeg

def update(change):
    global blocked_slider, robot
    new_image = change['new'] 
    
    # Run inference
    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    
    # Get x and y values from model output
    x = float(output[0]) if len(output) > 0 else 0.0
    y = float(output[1]) if len(output) > 1 else 0.0
    
    # Update widgets with model outputs
    network_output_slider.value = x
    x_output_label.value = f'X Value: {x:.4f}'
    y_output_label.value = f'Y Value: {y:.4f}'
    
    # Display inference results if camera is on
    if(state_widget.value == 'On'):
        # Convert normalized coordinates (-1 to 1) to pixel coordinates
        x_pixel = int(camera.width * (x / 2.0 + 0.5))
        y_pixel = int(camera.height * (y / 2.0 + 0.5))
        
        # Draw blue circle at the inferred position
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x_pixel, y_pixel), 8, (255, 0, 0), 3)
        
        # Add text showing the coordinates
        cv2.putText(prediction, f'X: {x:.3f}, Y: {y:.3f}', (10, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        prediction_widget.value = bgr8_to_jpeg(prediction)

# Initialize with first camera frame
update({'new': camera.value})

In [6]:
# Start observing camera frames
camera.observe(update, names='value')

In [7]:
# Start the camera
camera.running = True

## When you're done

In [8]:
# To stop the camera, run this cell
camera.running = False
print("Camera stopped")